In [8]:
import pandas as pd
import numpy as np
from glob import glob
from skfeature.function.similarity_based import fisher_score,SPEC,trace_ratio,lap_score
from skfeature.function.similarity_based import reliefF
from skfeature.function.statistical_based import f_score
from skfeature.function.information_theoretical_based import CIFE
import time
import warnings
warnings.filterwarnings("ignore")

In [9]:
info_file_list = glob("20210228_raw_anhui_data\\info\\*.csv")
info_dict = {}
# 读取收资信息
for info_file in info_file_list:
    print(info_file)
    csv_data = pd.read_csv(info_file,usecols=["datatime","real_power"])
    csv_data["datatime"] = pd.to_datetime(csv_data["datatime"],format="%d/%m/%Y %H:%M:%S")
    csv_data.index = csv_data["datatime"]
    csv_data.drop(["datatime"],axis=1,inplace=True)
    farm_name = info_file.split("\\")[-1].split("_")[0]
    farm_name = farm_name[:8]+"J"+farm_name[9:]
    info_dict[farm_name] = csv_data

20210228_raw_anhui_data\info\EEE09001P001_info.csv
20210228_raw_anhui_data\info\EEE09001P002_info.csv
20210228_raw_anhui_data\info\EEE09001P003_info.csv
20210228_raw_anhui_data\info\EEE09001P004_info.csv
20210228_raw_anhui_data\info\EEE09001P005_info.csv
20210228_raw_anhui_data\info\EEE09001P006_info.csv
20210228_raw_anhui_data\info\EEE09001P007_info.csv
20210228_raw_anhui_data\info\EEE09001P008_info.csv
20210228_raw_anhui_data\info\EEE09001P009_info.csv
20210228_raw_anhui_data\info\EEE09001P010_info.csv
20210228_raw_anhui_data\info\EEE09001P011_info.csv
20210228_raw_anhui_data\info\EEE09001P012_info.csv
20210228_raw_anhui_data\info\EEE09001P013_info.csv
20210228_raw_anhui_data\info\EEE09001P014_info.csv
20210228_raw_anhui_data\info\EEE09001P015_info.csv
20210228_raw_anhui_data\info\EEE09001P016_info.csv
20210228_raw_anhui_data\info\EEE09001P017_info.csv
20210228_raw_anhui_data\info\EEE09001P018_info.csv
20210228_raw_anhui_data\info\EEE09001P019_info.csv
20210228_raw_anhui_data\info\EE

In [16]:
# fisher score算法
def my_fisher_score(X,Y,name_dict,):
    NEED_NUM = 2000
    score = fisher_score.fisher_score(X.values[:NEED_NUM,:],Y.values[:NEED_NUM])
    rank_idx = fisher_score.feature_ranking(score)
    tmp_result = {}
    for i in range(len(rank_idx)):
        tmp_result[name_dict[rank_idx[i]]] = score[rank_idx[i]]
    return tmp_result
# spect算法
def my_spec(X,name_dict):
    score = SPEC.spec(X.values)
    rank_idx = SPEC.feature_ranking(score)
    tmp_result = {}
    for i in range(len(rank_idx)):
        tmp_result[name_dict[rank_idx[i]]] = score[rank_idx[i]]
    return tmp_result
# trace_ratio算法
def my_trace_ratio(X,Y,name_dict,FN=8):
    feature_idx,feature_score,subset_score = trace_ratio.trace_ratio(X,Y,FN)
    tmp_result = {}
    for i in range(FN):
        tmp_result[name_dict[feature_idx[i]]] = feature_score[i]
    return tmp_result
# reliefF算法
def my_reliefF(X,Y,name_dict):
    NEED_NUM = 2000
    score = reliefF.reliefF(X.values[:NEED_NUM,:],Y.values[:NEED_NUM])
    rank_idx = fisher_score.feature_ranking(score)
    tmp_result = {}
    for i in range(len(rank_idx)):
        tmp_result[name_dict[rank_idx[i]]] = score[rank_idx[i]]
    return tmp_result
#f_score算法
def my_f_score(X,Y,name_dict):
    NEED_NUM = 1000000000
    score = f_score.f_score(X.values[:NEED_NUM,:],Y.values[:NEED_NUM])
    rank_idx = fisher_score.feature_ranking(score)
    tmp_result = {}
    for i in range(len(rank_idx)):
        tmp_result[name_dict[rank_idx[i]]] = score[rank_idx[i]]
    return tmp_result
# lap_score算法
def my_lap_score(X,name_dict):
    score = lap_score.lap_score(X.values)
    rank_idx = lap_score.feature_ranking(score)
    tmp_result = {}
    for i in range(len(rank_idx)):
        tmp_result[name_dict[rank_idx[i]]] = score[rank_idx[i]]
    return tmp_result
# CIFE算法
def my_CIFE(X,Y,name_dict):
    NEED_NUM = 1000000000
    score = CIFE.cife(X.values[:NEED_NUM,:],Y.values[:NEED_NUM],n_selected_features = 6)
    print(score)
    good_index = score[0]
    good_score = score[1]
    rank_idx = fisher_score.feature_ranking(score)
    tmp_result = {}
    for i in range(len(good_index)):
        tmp_result[name_dict[good_index[i]]] = good_score[i]
    return tmp_result

In [19]:
no_need_farm=["EEE09001J035","EEE09001J036","EEE09001J037","EEE09001J038"]
EACH_CSV_NUM = 4000
all_weather_score_dict ={}
# 合并不同气象的特征，然后进行特征筛选工作
for weather in ["W001","W002","W003","W005","W006"]:
    file_list = glob("20210228_raw_anhui_data\\anhui_wind_{}\\*.csv".format(weather))
    for counter, file in enumerate(file_list):
        farm_name = file.split("\\")[-1].split("_")[0]
        if(farm_name in no_need_farm):
            continue
        csv_data = pd.read_csv(file)
        csv_data["DateTime"] = pd.to_datetime(csv_data["DateTime"],format="%Y-%m-%d %H:%M:%S")
        csv_data.index = csv_data["DateTime"]
        csv_data.drop(["DateTime"],axis=1,inplace=True)
        csv_data = pd.concat([csv_data,info_dict[farm_name]],join="inner",axis=1)
        # 过滤-99
        need_idx  = csv_data["real_power"]!=-99
        csv_data = csv_data.loc[need_idx,:]
        if counter==0:
            all_farm_data = csv_data.iloc[:EACH_CSV_NUM,:]
        else:
            all_farm_data = all_farm_data.append(csv_data.iloc[:EACH_CSV_NUM,:])
        break
    print("{}:{}".format(farm_name,np.shape(all_farm_data)))
    # 收资信息
    Y = all_farm_data["real_power"]
    all_farm_data.drop(["real_power"],axis=1,inplace=True)
    # 气象信息
    X = all_farm_data
    name_dict = {}
    for i in range(len(all_farm_data.columns)):
        name_dict[i] = all_farm_data.columns[i]
    t1 = time.time()
    #tmp_result = my_trace_ratio(X,Y,name_dict)
    # 调用特征筛选算法
    tmp_result = my_CIFE(X,Y,name_dict)
    print("{} cost:{}".format(weather,time.time()-t1))
    all_weather_score_dict[weather] = tmp_result
    print("finish {}".format(weather))

EEE09001J001:(4000, 37)
(array([26,  0, 33, 19, 18, 17], dtype=int64), array([ 9.64390707e+00,  1.77635684e-15, -9.61534091e+00, -1.92218705e+01,
       -2.88379663e+01, -3.84588734e+01]), array([9.64390707, 9.63990707, 9.61934091, 9.62521835, 9.62840707,
       9.63040707]))
W001 cost:5.963098764419556
finish W001
EEE09001J001:(4000, 37)
(array([ 6, 23, 27, 21, 26, 22], dtype=int64), array([ 9.64290707e+00,  1.00000000e-03, -5.73004674e+00, -1.14717642e+01,
       -1.73179580e+01, -2.31703390e+01]), array([9.64290707, 7.65601841, 7.64245705, 7.66350258, 7.63917439,
       7.66092309]))
W002 cost:5.532236337661743
finish W002
EEE09001J001:(4000, 40)
(array([ 4, 14, 29, 27, 25, 26], dtype=int64), array([ 9.37786461e+00,  1.00000000e-03, -4.77368696e+00, -1.23087802e+01,
       -1.98917243e+01, -2.75128735e+01]), array([9.37786461, 9.35733818, 4.79071339, 8.3974344 , 8.45918591,
       8.46359991]))
W003 cost:6.2200164794921875
finish W003
EEE09001J001:(4000, 36)
(array([ 0,  1, 10, 11, 

In [20]:
# 打印出每个气象中的特征重要性分值，特征越重要分值越高
for weather in ["W001","W002","W003","W005","W006"]:
    counter = 0
    csv_data = []
    print("{}========================".format(weather))
    for k,v in all_weather_score_dict[weather].items():
        counter+=1
        print("{}  {}  {}".format(counter,k,v))
        csv_data.append([k,v])
    pd.DataFrame(csv_data).to_csv("{}.csv".format(weather))

W001========================
1  RHU_112  9.64390707258672
2  UWIND_001  1.7763568394002505e-15
3  AHU_120  -9.615340906960103
4  TMP_112  -19.221870535795738
5  TMP_106  -28.837966330258002
6  TMP_103  -38.458873402844745
W002========================
1  UWIND_217  9.64290707258672
2  SHU_205  0.0010000000000012221
3  SHU_217  -5.730046735233335
4  SHU_201  -11.471764234297032
5  SHU_214  -17.31795804488102
6  SHU_203  -23.170339021120853
W003========================
1  UWIND_103  9.377864608345547
2  VWIND_112  0.0010000000000029985
3  VIS_000  -4.773686962548378
4  RHU_112  -12.308780188257797
5  RHU_103  -19.891724316093736
6  RHU_106  -27.512873501890105
W005========================
1  UWIND_001  9.446694917657878
2  UWIND_100  0.0
3  VWIND_106  -9.446194917657879
4  VWIND_112  -18.891889835315766
5  VWIND_120  -28.337584752973648
6  PRES_000  -37.78327967063153
W006========================
1  UWIND_102  9.445441151711313
2  UWIND_001  0.0
3  SSRD_000  -4.925477775886886
4  UWIND_10